# llm_client

> Fill in a module description here

In [ ]:
#| default_exp llm_client

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
import time
from typing import Dict, Any, Optional
import requests
from llm_data_extractor.models import LLMConfig

In [ ]:
#| export


class LLMClientError(Exception):
    """Custom exception for LLM client errors."""
    pass


def get_llm_response(prompt: str, config: LLMConfig) -> str:
    """
    Get response from LLM API with retry logic.
    
    Args:
        prompt: The prompt to send to the LLM
        config: LLM configuration object
        
    Returns:
        Raw response string from the LLM
        
    Raises:
        LLMClientError: If all retry attempts fail
    """
    
    for attempt in range(config.max_retries):
        try:
            if "gpt" in config.model_name.lower() or "openai" in config.model_name.lower():
                return _call_openai_api(prompt, config)
            elif "claude" in config.model_name.lower():
                return _call_anthropic_api(prompt, config)
            elif _is_huggingface_model(config.model_name):
                return _call_huggingface_api(prompt, config)
            else:
                return _call_generic_api(prompt, config)
                
        except Exception as e:
            if attempt == config.max_retries - 1:
                raise LLMClientError(f"Failed after {config.max_retries} attempts: {str(e)}")
            
            # Exponential backoff
            time.sleep(2 ** attempt)
    
    raise LLMClientError("Unexpected error in retry loop")


def _call_openai_api(prompt: str, config: LLMConfig) -> str:
    """Call OpenAI API."""
    try:
        import openai
    except ImportError:
        raise LLMClientError("openai package not installed. Run: pip install openai")
    
    # Set up client
    client_kwargs = {}
    if config.api_key:
        client_kwargs["api_key"] = config.api_key
    if config.base_url:
        client_kwargs["base_url"] = config.base_url
        
    client = openai.OpenAI(**client_kwargs)
    
    # Prepare request
    messages = [{"role": "user", "content": prompt}]
    request_params = {
        "model": config.model_name,
        "messages": messages,
        "temperature": config.temperature,
        "timeout": config.timeout,
    }
    
    if config.max_tokens:
        request_params["max_tokens"] = config.max_tokens
    
    # Make request
    response = client.chat.completions.create(**request_params)
    return response.choices[0].message.content


def _call_anthropic_api(prompt: str, config: LLMConfig) -> str:
    """Call Anthropic API."""
    try:
        import anthropic
    except ImportError:
        raise LLMClientError("anthropic package not installed. Run: pip install anthropic")
    
    client_kwargs = {}
    if config.api_key:
        client_kwargs["api_key"] = config.api_key
    if config.base_url:
        client_kwargs["base_url"] = config.base_url
        
    client = anthropic.Anthropic(**client_kwargs)
    
    request_params = {
        "model": config.model_name,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": config.temperature,
        "timeout": config.timeout,
    }
    
    if config.max_tokens:
        request_params["max_tokens"] = config.max_tokens
    
    response = client.messages.create(**request_params)
    return response.content[0].text


def _call_generic_api(prompt: str, config: LLMConfig) -> str:
    """Generic API call for other providers."""
    if not config.base_url:
        raise LLMClientError("base_url required for generic API calls")
    
    headers = {
        "Content-Type": "application/json",
    }
    
    if config.api_key:
        headers["Authorization"] = f"Bearer {config.api_key}"
    
    payload = {
        "model": config.model_name,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": config.temperature,
    }
    
    if config.max_tokens:
        payload["max_tokens"] = config.max_tokens
    
    response = requests.post(
        f"{config.base_url.rstrip('/')}/chat/completions",
        headers=headers,
        json=payload,
        timeout=config.timeout
    )
    
    response.raise_for_status()
    
    result = response.json()
    return result["choices"][0]["message"]["content"]


def _is_huggingface_model(model_name: str) -> bool:
    """Check if model name indicates a Hugging Face model."""
    hf_indicators = [
        "meta-llama", "mistralai", "microsoft", "google", "qwen", 
        "codellama", "mixtral", "phi", "gemma", "llama", "mistral"
    ]
    return any(indicator in model_name.lower() for indicator in hf_indicators)


def _call_huggingface_api(prompt: str, config: LLMConfig) -> str:
    """Call Hugging Face Router API using OpenAI-compatible format."""


    API_URL = "https://router.huggingface.co/v1/chat/completions"
        
    if not config.api_key:
        raise LLMClientError("Hugging Face API token required. Set api_key in LLMConfig.")
    
    headers = {
        "Authorization": f"Bearer {config.api_key}",
        "Content-Type": "application/json",
    }
    
    # Prepare payload using OpenAI chat completions format
    payload = {
        "messages": [
            {
                "role": "user", 
                "content": prompt
            }
        ],
        "model": config.model_name,
        "temperature": config.temperature,
    }
    
    #print('\nPayload:',payload)

    if config.max_tokens:
        payload["max_tokens"] = config.max_tokens
    
    try:
        response = requests.post(
            API_URL,
            headers=headers,
            json=payload,
            timeout=config.timeout
        )
        
        response.raise_for_status()
        result = response.json()
        print('\nResponse:', result)
        
        # Extract content from OpenAI-style response
        if "choices" in result and len(result["choices"]) > 0:
            choice = result["choices"][0]
            if "message" in choice:
                return choice["message"]["content"]
            elif "text" in choice:
                return choice["text"]
        
        # Fallback for unexpected format
        raise LLMClientError(f"Unexpected response format: {result}")
            
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 503:
            raise LLMClientError("Model is loading on Hugging Face. Please wait and retry.")
        elif e.response.status_code == 429:
            raise LLMClientError("Rate limit exceeded. Please wait before retrying.")
        elif e.response.status_code == 401:
            raise LLMClientError("Invalid Hugging Face token. Check your API key.")
        else:
            error_text = ""
            try:
                error_detail = e.response.json()
                error_text = error_detail.get("error", {}).get("message", str(error_detail))
            except:
                error_text = e.response.text
            raise LLMClientError(f"HTTP error {e.response.status_code}: {error_text}")
    except requests.exceptions.RequestException as e:
        raise LLMClientError(f"Request failed: {str(e)}")


def parse_llm_json_response(response: str) -> Dict[str, Any]:
    """
    Parse JSON response from LLM, handling common formatting issues.
    
    Args:
        response: Raw response string from LLM
        
    Returns:
        Parsed JSON object
        
    Raises:
        LLMClientError: If JSON cannot be parsed
    """
    # Clean up common issues
    cleaned_response = response.strip()
    
    # Remove markdown code block markers if present
    if cleaned_response.startswith("```json"):
        cleaned_response = cleaned_response[7:]
    if cleaned_response.startswith("```"):
        cleaned_response = cleaned_response[3:]
    if cleaned_response.endswith("```"):
        cleaned_response = cleaned_response[:-3]
    
    cleaned_response = cleaned_response.strip()
    
    try:
        return json.loads(cleaned_response)
    except json.JSONDecodeError as e:
        raise LLMClientError(f"Failed to parse JSON response: {str(e)}\nResponse: {response[:200]}...")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()